In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
from queue import Queue
from IPython.display import clear_output

In [2]:
tf.__version__

'2.9.1'

# Create a Submission File 

The submitted file must include all the predictions from Dan, Kyu, and Play Style testing CSVs. Therefore, you must train your models before submitting any file. If your model is not trained for Dan, Kyu, or Play Style, check out the previous tutorials about how to train your model or just simply use the baseline models (model_dan_tutorial.h5, model_kyu_tutorial.h5, model_playstyle.h5) 

**PublicUpload.csv** must be in the following form:
```
DTPU0000000001,id,qr,pq,pd,ab
DTPU0000000002,ao,ab,ha,ff,qd
DTPU0000000003,qd,gd,fh,ed,fa
DTPU0000000004,pr,ba,dq,hh,aj
DTPU0000000005,ph,jh,af,df,gj
.
.
.
KTPU0000000001,id,qr,pq,pd,ab
KTPU0000000002,ao,ab,ha,ff,qd
KTPU0000000003,qd,gd,fh,ed,fa
KTPU0000000004,pr,ba,dq,hh,aj
KTPU0000000005,ph,jh,af,df,gj
.
.
.
PSTPU0000000001_79,2
PSTPU0000000001_55,3
PSTPU0000000001_105,2
PSTPU0000000002_37,1
PSTPU0000000002_113,3
```

**For Dan and Kyu:**
- Column 1: Game ID
- Column 2: Predicted Moves, up to 5 predictions for each game

**For PlayStyle**:
- Column 1: Game ID
- Column 2: Predicted Game Style

The code block below is copied from the tutorial notebooks and it includes the pre-processing methods of the trained models. You might need to change the functions below with respect to your models.

In [3]:
liber = Queue()
def count_air2(move):
    bfs = []
    col = coordinates[move[2]]
    row = coordinates[move[3]]
    #找四周要bfs的
    bfs.append([row, col])
    if col - 1 >= 0 and x[row][col - 1][8] == 0:
        bfs.append([row, col - 1])
    if col + 1 < 19 and x[row][col + 1][8] == 0:
        bfs.append([row, col + 1])
    if row - 1 >= 0 and x[row - 1][col][8] == 0:
        bfs.append([row - 1, col])
    if row + 1 < 19 and x[row + 1][col][8] == 0:
        bfs.append([row + 1, col])
    for m in bfs:
        BFS(m)
    bfs = []
    if not liber.empty():
#         print("liber")
        while not liber.empty():
            li_row, li_col = liber.get()
#             x[li_row][li_col][8] = 1
            if li_col - 1 >= 0 and x[li_row][li_col - 1][8] == 0:
                bfs.append([li_row, li_col - 1])
            if li_col + 1 < 19 and x[li_row][li_col + 1][8] == 0:
                bfs.append([li_row, li_col + 1])
            if li_row - 1 >= 0 and x[li_row - 1][li_col][8] == 0:
                bfs.append([li_row - 1, li_col])
            if li_row + 1 < 19 and x[li_row + 1][li_col][8] == 0:
                bfs.append([li_row + 1, li_col])
        for m in bfs:
            BFS(m)
        
def BFS(now):
    q = Queue()
    row = now[0]
    col = now[1]
    q.put((row, col))
    visited = [[False for _ in range(19)] for _ in range(19)]
    nodes = []
    air = 0
    next_board = 0
    board = player
    if oppnent[row, col] == 1:
        board = oppnent
        next_board = 4
    while not q.empty():
        m, n = q.get()
        if m < 0 or n < 0 or m >= 19 or n >= 19 or visited[m][n]:
            continue
        visited[m][n] = True
        if board[m, n] == 1:
            nodes.append([m, n])
            q.put((m - 1, n))
            q.put((m + 1, n))
            q.put((m, n - 1))
            q.put((m, n + 1))
        elif x[m, n, 8] == 1: #空地 -> 自由度+1
            air += 1
    dele = -1
    for i in range(8):
        if x[row][col][i] == 1:
            dele = i
            break
    air = min(air, 4)
    for node in nodes:
        n_row = node[0]
        n_col = node[1]
        #原本的改0，現在的氣數對應的棋盤改1
        if dele != -1:
            x[n_row][n_col][dele] = 0;
        if air == 0:
            x[n_row][n_col][8] = 1
            x[n_row, n_col, 17] = 0
            x[n_row, n_col, 18] = 0
            player[n_row, n_col] = 0
            oppnent[n_row, n_col] = 0
            liber.put(node)
        else:
            x[n_row][n_col][next_board + air - 1] = 1

In [4]:
chars = 'abcdefghijklmnopqrs'
coordinates = {k:v for v,k in enumerate(chars)}
chartonumbers = {k:v for k,v in enumerate(chars)}

x = np.zeros((19,19,19))
player = np.zeros((19, 19))
oppnent = np.zeros((19, 19))

def prepare_input_for_dan_kyu_models(moves, player_color):
    #     player_color = moves[-1][0] #
    sz = len(moves)
    if sz == 0:
        return x;
    move = moves[-1]
    
    color = move[0]
    column = coordinates[move[2]]
    row = coordinates[move[3]]
    if color == player_color: #
        player[row,column] = 1
    else: #
        oppnent[row,column] = 1
    x[row,column,8] = 0
    
    #倒數8步
    for i in range(16, 9, -1):
        x[:, :, i] = x[:, :, i - 1]
#     x[:, :, 10:16] = np.copy(x[:, :, 9:15])
    x[:, :, 9] = 0
#     x[:, :, 9] = np.zeros((19, 19))
    x[row, column, 9] = 1
#     for i in range(8):
#         if sz - 1 - i - 1 >= 0:
#             col_l = coordinates[moves[sz - 1 - i - 1][2]]
#             row_l = coordinates[moves[sz - 1 - i - 1][3]]
#             x[row_l, col_l, 9 + i] = 0
#             if i != 7:
#                 x[row_l, col_l, 9 + i + 1] = 1
#     x[row, column, 9] = 1
    #周圍7*7
    x[:, :, 17] = 0
#     x[:, :, 17] = np.zeros((19, 19))
    x[:, :, 18] = 0
#     x[:, :, 18] = np.zeros((19, 19))
    last_col = coordinates[move[2]]
    last_row = coordinates[move[3]]
    rad = 3 #要改範圍大小的話改這個
    row1 = max(0, last_row - rad)
    row7 = min(18, last_row + rad)
    col1 = max(0, last_col - rad)
    col7 = min(18, last_col + rad)
    for i in range(row1, row7 + 1, 1):
        for j in range(col1, col7 + 1, 1):
            x[i, j, 17] = player[i, j]
            x[i, j, 18] = oppnent[i, j]
    
    count_air2(move)
    return x

def prepare_input_for_playstyle_model(moves, player_color):
#     player_color = moves[-1][0] #
    sz = len(moves)
#     if sz == 0:
#         return x;
    move = moves[-1]
    
    color = move[0]
    column = coordinates[move[2]]
    row = coordinates[move[3]]
    if color == player_color: #
        player[row,column] = 1
    else: #
        oppnent[row,column] = 1
    x[row,column,8] = 0
    
    #倒數8步
    for i in range(16, 9, -1):
        x[:, :, i] = x[:, :, i - 1]
#     x[:, :, 10:16] = np.copy(x[:, :, 9:15])
    x[:, :, 9] = 0
#     x[:, :, 9] = np.zeros((19, 19))
    x[row, column, 9] = 1
#     for i in range(8):
#         if sz - 1 - i - 1 >= 0:
#             col_l = coordinates[moves[sz - 1 - i - 1][2]]
#             row_l = coordinates[moves[sz - 1 - i - 1][3]]
#             x[row_l, col_l, 9 + i] = 0
#             if i != 7:
#                 x[row_l, col_l, 9 + i + 1] = 1
#     x[row, column, 9] = 1
    #周圍
    x[:, :, 17:23] = 0
    last_col = coordinates[move[2]]
    last_row = coordinates[move[3]]
    for rad in range(3, 0, -1):
        row1 = max(0, last_row - rad)
        row7 = min(18, last_row + rad)
        col1 = max(0, last_col - rad)
        col7 = min(18, last_col + rad)
#         print(row1, row7, col1, col7)
        for i in range(row1, row7 + 1, 1):
            for j in range(col1, col7 + 1, 1):
                x[i, j, 17 + (3 - rad) * 2] = player[i, j]
                x[i, j, 18 + (3 - rad) * 2] = oppnent[i, j]
    
    count_air2(move)
#     for i in range(8, 9):
#         print("Case ", end = '')
#         print(i)
#         for j in range(0, 19):
#             for k in range(0, 19):
#                 if x[j][k][i] == 1:
#                     print(chartonumbers[k], chartonumbers[j])
#                 print(int(x[j][k][i]), end = ' ')
#             print()
#         print()
    
    #列印所有棋盤
#     for i in range(0, 3, 1):
#         print("  a b c d e f g h i j k l m n o p q r s")
#         for j in range(0, 19, 1):
#             print(chars[j], end = " ")
#             for k in range(0, 19, 1):
#                 print(int(x[j, k, i]), end = " ")
#             print("")
#         print("")
    
    return x

def number_to_char(number):
    number_1, number_2 = divmod(number, 19)
    return chartonumbers[number_1] + chartonumbers[number_2]

def top_5_preds_with_chars(predictions):
    resulting_preds_numbers = [np.flip(np.argpartition(prediction, -5)[-5:]) for prediction in predictions]
    resulting_preds_chars = np.vectorize(number_to_char)(resulting_preds_numbers)
    return resulting_preds_chars

## Save Predictions for Dan
The code block below is to use **dan model** to predict and save the results in required form. It generates the best 5 predictions for each sample and convert them to character coordinates.

In [5]:
# Load your own model. Here we use the baseline model
model = load_model('./model_dan_one_epoch.h5')

# Load the corresponding dataset
df = open('./CSVs/dan_test_public.csv').read().splitlines()
games_id = [i.split(',',2)[0] for i in df]
games = [i.split(',',2)[-1] for i in df]
colors = [i.split(',',2)[1] for i in df]

x_testing = []

for game_i, game in enumerate(games):
    moves_list = game.split(',')
    x_testing.append(prepare_input_for_dan_kyu_models(moves_list, colors[game_i]))

x_testing = np.array(x_testing)
predictions = model.predict(x_testing)
prediction_chars = top_5_preds_with_chars(predictions)

# Save results to public_submission_template.csv
with open('./public_submission_template.csv','a') as f:
    for index in range(len(prediction_chars)):
        answer_row = games_id[index] + ',' + ','.join(prediction_chars[index]) + '\n'
        f.write(answer_row)

OSError: No file or directory found at ./model_dan_one_epoch.h5

## Save Predictions for Kyu
The same code block applies to Kyu by changing the model and dataset

In [5]:
# Load your own model. Here we use the baseline model
model = load_model('./models/model_kyu_v2_b64_f256_l49_06_0.5309.h5')

# Load the corresponding dataset
df = open('./CSVs/kyu_test_public.csv').read().splitlines()
games_id = [i.split(',',2)[0] for i in df]
games = [i.split(',',2)[-1] for i in df]
colors = [i.split(',',2)[1] for i in df]

x_testing = []

for game_i, game in enumerate(games):
    print(game_i, end=' ')
    moves_list = game.split(',')
    pi = []
    x = np.zeros((19,19,19))
    player = np.zeros((19, 19))
    oppnent = np.zeros((19, 19))
    x[:, :, 8] = np.ones((19, 19))
    for count, m in enumerate(moves_list):
        pi = (prepare_input_for_dan_kyu_models(moves_list[:count+1], colors[game_i]))
    clear_output(wait=True)
    x_testing.append(pi.copy())

x_testing = np.array(x_testing)
predictions = model.predict(x_testing)
prediction_chars = top_5_preds_with_chars(predictions)

# Save results to public_submission_template.csv
with open('./public_submission_template.csv','a') as f:
    for index in range(len(prediction_chars)):
        answer_row = games_id[index] + ',' + ','.join(prediction_chars[index]) + '\n'
        f.write(answer_row)

355/355 [==============================] - 22s 39ms/step


## Save Predictions for PlayStyle
Playstyle requires only one predictions so the code here is a simple version of above

In [ ]:
# Load your own model. Here we use the baseline model
model = load_model('./models/model_style_v2_b32_f256_l148_tra095_rot_02_20_0.7371.h5')

# Load the corresponding dataset
df = open('./CSVs/play_style_test_public.csv').read().splitlines()
games_id = [i.split(',',2)[0] for i in df]
games = [i.split(',',2)[-1] for i in df]

x_testing = []

for game_i, game in enumerate(games):
    clear_output(wait=True)
    print(game_i, end=' ')
    x = np.zeros((19,19,23))
    player = np.zeros((19, 19))
    oppnent = np.zeros((19, 19))
#         x[:, :, 8] = 1
    x[:, :, 8] = np.ones((19, 19))
    moves_list = game.split(',')
    for count, m in enumerate(moves_list):
        pi = (prepare_input_for_playstyle_model(moves_list[:count+1], moves_list[-1][0]))
    x_testing.append(pi.copy())

x_testing = np.array(x_testing)
predictions = model.predict(x_testing)
prediction_numbers = np.argmax(predictions, axis=1)
# print(np.shape(x_testing))
with open('./public_submission_template.csv','a') as f:
    for index, number in enumerate(prediction_numbers):
        answer_row = games_id[index] + ',' + str(number+1) + '\n'
        f.write(answer_row)

4762 

Now you can upload your file from the competition website! (https://tbrain.trendmicro.com.tw/Competitions/Details/29)